## Restructuring csv file  

### General csv info
- There are 39,900+ songs
- There are a columns that are not important
- Duplicates make 9812 records and 5368 unique song names

### Process Outline
- Create column with unique identifiers
- Analyze duplicate song titles
- Create additional summary statistics per song
    - ###
- Validate if weeks on chart matches weekly record
    - does the difference belong due times when the song is on the top 100 after week 92+
- Restructure song records from row to column structure for easier plotting


Find songs with the same year and Rank = typically should be the same song with different collaborators
Find song name duplicates - could be a re-release in a different year or could be a cover by a different artist



In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
plt.rcParams["figure.figsize"] = (16,9)
pd.set_option('display.max_columns', 30)

In [13]:
# determined encodings with `file --exclude encoding Billboard.csv`
df = pd.read_csv("./data/Billboard.csv", low_memory=False, encoding="ISO-8859-1", parse_dates=['Date Entered', 'Date Peaked'])
df.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,Milky Chance,Milky Chance,NaN,NaN,Stolen Dance - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,Fall Out Boy,Fall Out Boy,NaN,NaN,American Beauty / American Psycho,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,Nick Jonas,"Jonas, Nick",NaN,NaN,Nick Jonas (Deluxe Version),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,Meghan Trainor,"Trainor, Meghan",NaN,NaN,Title - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,Florida Georgia Line,Florida Georgia Line,NaN,NaN,Anything Goes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
# check data quality
df.dtypes.head(30)

Year                 int64
Yearly Rank         object
Source              object
Prefix              object
CH                 float64
40                 float64
10                 float64
PK                 float64
High                object
Verified            object
Artist              object
Artist Inverted     object
Featured            object
UnFeatured          object
Album               object
B-Side              object
Track               object
Time                object
Time Source         object
explicit            object
Time (Album)        object
Artist ID          float64
SYMBL               object
ReIssue             object
Label/Number        object
Media               object
Stereo (55-68)      object
Pic Sleeve          object
Genre               object
Comments            object
dtype: object

In [15]:
# Fix Yearly Rank so 'n/a' text values are labled as -1
df['Yearly Rank'].replace('n/a', int(-1), inplace=True);
df['Yearly Rank'] = df['Yearly Rank'].fillna(-1);
df['High'].replace([None, '--'], -1, inplace=True);
pd.to_datetime(df['Time'], format='%M:%S');

In [16]:
df['Date Entered'].replace("+", "", inplace=True)
df['Date Peaked'].replace("+", "", inplace=True)

In [17]:
df['Date Entered'].value_counts().tail(70)


2020-01-27    1
2020-06-12    1
2019-12-27    1
1919-02-22    1
2019-07-19    1
             ..
2015-05-08    1
2015-02-25    1
2015-04-30    1
2014-02-26    1
1890-10-25    1
Name: Date Entered, Length: 70, dtype: int64

In [7]:
# Conversion not needed
# df['Date Entered'] = pd.to_datetime(df['Date Entered'], format='%m/%d/%Y')

In [19]:
df[ ~df['Date Entered'].apply(lambda x: isinstance(x, datetime))]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week


In [95]:
df['Yearly Rank'].value_counts().tail(30)

290b    1
307b    1
314b    1
492b    1
509b    1
35a     1
288a    1
204b    1
164b    1
165b    1
48b     1
43b     1
111b    1
53b     1
59b     1
67b     1
78b     1
118b    1
171b    1
305b    1
286b    1
325b    1
418b    1
11b     1
49a     1
33b     1
33a     1
28b     1
201b    1
146b    1
Name: Yearly Rank, dtype: int64

In [20]:
df[ ~df['Yearly Rank'].apply(lambda x: np.isreal(x))]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,Milky Chance,Milky Chance,NaN,NaN,Stolen Dance - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,Fall Out Boy,Fall Out Boy,NaN,NaN,American Beauty / American Psycho,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,Nick Jonas,"Jonas, Nick",NaN,NaN,Nick Jonas (Deluxe Version),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,Meghan Trainor,"Trainor, Meghan",NaN,NaN,Title - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,Florida Georgia Line,Florida Georgia Line,NaN,NaN,Anything Goes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39977,1891,12,m,1891_012,3.0,NaN,NaN,3.0,2,NaN,Dan Kelly,"Kelly, Dan",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39978,1891,13,m,1891_013,2.0,NaN,NaN,2.0,2,NaN,Dan Kelly,"Kelly, Dan",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39979,1890,1,m,1890_001,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39980,1890,2,m,1890_002,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df[ ~df['Yearly Rank'].apply(lambda x: isinstance(x, (int, float)))]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,Milky Chance,Milky Chance,NaN,NaN,Stolen Dance - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,Fall Out Boy,Fall Out Boy,NaN,NaN,American Beauty / American Psycho,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,Nick Jonas,"Jonas, Nick",NaN,NaN,Nick Jonas (Deluxe Version),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,Meghan Trainor,"Trainor, Meghan",NaN,NaN,Title - EP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,Florida Georgia Line,Florida Georgia Line,NaN,NaN,Anything Goes,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39977,1891,12,m,1891_012,3.0,NaN,NaN,3.0,2,NaN,Dan Kelly,"Kelly, Dan",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39978,1891,13,m,1891_013,2.0,NaN,NaN,2.0,2,NaN,Dan Kelly,"Kelly, Dan",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39979,1890,1,m,1890_001,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39980,1890,2,m,1890_002,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# Maybe make this a separate column
df['Yearly Rank'] = df['Yearly Rank'].replace('^[A-Za-z]+$', '', regex=True, inplace=True)

In [23]:
df[df['Yearly Rank'].str.contains('[A-Za-z]', na=False)]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week


In [24]:
df.dtypes

Year             int64
Yearly Rank     object
Source          object
Prefix          object
CH             float64
                ...   
87th Week      float64
88th Week      float64
89th Week      float64
90th Week      float64
91st Week      float64
Length: 125, dtype: object

In [25]:
df['Yearly Rank'] = pd.to_numeric(df['Yearly Rank'])

In [26]:
df.dtypes

Year             int64
Yearly Rank    float64
Source          object
Prefix          object
CH             float64
                ...   
87th Week      float64
88th Week      float64
89th Week      float64
90th Week      float64
91st Week      float64
Length: 125, dtype: object

In [27]:
df[ ~df['Yearly Rank'].apply(lambda x: isinstance(x, (int, float)))]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,Featured,UnFeatured,Album,...,77th Week,78th Week,79th Week,80th Week,81st Week,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week


In [28]:
# Look at basic statistics
df.iloc[:, :50].describe()

,Year,Yearly Rank,CH,40,10,PK,Artist ID
count,39982.000000,0.0,39929.000000,27942.000000,27832.000000,30358.000000,2302.000000
mean,1967.166575,NaN,9.274162,4.523549,1.157445,1.545523,7570.439618
std,27.458659,NaN,7.564290,6.682420,3.088947,1.118186,1278.816779
min,1890.000000,NaN,0.000000,0.000000,0.000000,0.000000,1172.000000
25%,1949.000000,NaN,3.000000,0.000000,0.000000,1.000000,7665.000000
50%,1968.000000,NaN,7.000000,0.000000,0.000000,1.000000,8043.000000
75%,1988.000000,NaN,14.000000,8.000000,0.000000,2.000000,8167.750000
max,2015.000000,NaN,87.000000,63.000000,32.000000,17.000000,8445.000000


In [213]:
# identify and store track name of duplicates
single_items = []
duplicates = []
for x in df['Track'].values:
    if x in single_items:
        duplicates.append(x)
    else:
        single_items.append(x)
        
len(duplicates)

9812

In [214]:
# Number of duplicated songs (unique)
len(set(duplicates))

5368

In [225]:
df['Primary_Artist'] = df[df['Track'].isin(list(set(duplicates)))]['Artist'].apply(lambda x: str(x).split(",")[0])
df['Primary_Artist'] = df['Primary_Artist'].replace({np.nan: ""})

def text_format(col1, col2):
    if col2 != "":
        text = str(col1) +  " - " + str(col2)
    else: 
        text = str(col1)
    
    return text

df['Track_Adjusted'] = df.apply(lambda x: text_format(x['Track'], x['Primary_Artist']), axis=1)


<IPython.core.display.Javascript object>

In [226]:
df

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,...,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253,Unnamed: 254,Unnamed: 255,Primary_Artist,Track_Adjusted
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,Milky Chance,Milky Chance,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Stolen Dance
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,Fall Out Boy,Fall Out Boy,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Centuries
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,Nick Jonas,"Jonas, Nick",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Nick Jonas,Jealous - Nick Jonas
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,Meghan Trainor,"Trainor, Meghan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Dear Future Husband
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,Florida Georgia Line,Florida Georgia Line,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Sun Daze
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39977,1891,12,m,1891_012,3.0,NaN,NaN,3.0,2,NaN,Dan Kelly,"Kelly, Dan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Pat Kelly on a Spree
39978,1891,13,m,1891_013,2.0,NaN,NaN,2.0,2,NaN,Dan Kelly,"Kelly, Dan",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Pat Kelly's Plea in His Own Defense
39979,1890,1,m,1890_001,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U.S. Marine Band,Semper Fidelis - U.S. Marine Band
39980,1890,2,m,1890_002,6.0,NaN,NaN,6.0,1,NaN,U.S. Marine Band,U.S. Marine Band,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,Washington Post


In [197]:
df['Artist'].apply(lambda x: str(x).split(",")[0])

0                Milky Chance
1                Fall Out Boy
2                  Nick Jonas
3              Meghan Trainor
4        Florida Georgia Line
                 ...         
39977               Dan Kelly
39978               Dan Kelly
39979        U.S. Marine Band
39980        U.S. Marine Band
39981        U.S. Marine Band
Name: Artist, Length: 39982, dtype: object

In [189]:
# do two analysis
# 1) only about repeats and their trends
# 2) repeats encoded to have different titles

df[ df['Track'] == 'White Christmas' ]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,Artist,Artist Inverted,...,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week,First_Week_on_Charts,Number_Week_on_Charts,diff
24105,1962,286,a,1962_286,3.0,1.0,0.0,1.0,38,x,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,0.0
24365,1962,546,a,1962_546,1.0,0.0,0.0,1.0,88,p,"Drifters, The","Drifters, The",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0
24620,1961,125,a,1961_125,4.0,3.0,0.0,1.0,12,x,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,4,0.0
25366,1960,188,a,1960_188,3.0,2.0,0.0,1.0,26,x,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,3,0.0
25758,1960,578,a,1960_578,1.0,0.0,0.0,1.0,96,x,"Drifters, The","Drifters, The",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0
26128,1959,347,a,1959_347,1.0,0.0,0.0,1.0,59,NaN,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0
26744,1958,382,Hot,1958_382,2.0,0.0,0.0,1.0,66,NaN,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,2,0.0
27130,1957,230,Top,1957_230,6.0,1.0,0.0,1.0,34,NaN,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,6,0.0
27740,1956,353,Top,1956_353,1.0,0.0,0.0,1.0,65,NaN,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,0.0
27952,1955,61,DJ,1955_061,3.0,2.0,1.0,1.0,7,p,Bing Crosby,"Crosby, Bing",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,-2.0


In [3]:
cols_keep = df.columns[:125]

In [4]:
df = df[cols_keep]
df.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
week_columns = [ x for x in cols_keep if "Week" in x]

In [33]:
df["Number_Week_on_Charts"] = df[week_columns].count(axis=1)
df

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week,First_Week_on_Charts,Number_Week_on_Charts
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25,25
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34,34
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32,32
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,16
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39977,1891,12,m,1891_012,3.0,NaN,NaN,3.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
39978,1891,13,m,1891_013,2.0,NaN,NaN,2.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
39979,1890,1,m,1890_001,6.0,NaN,NaN,6.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
39980,1890,2,m,1890_002,6.0,NaN,NaN,6.0,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0


In [58]:
df[ df['Number_Week_on_Charts'] != df['CH'] ]['Year'].value_counts().index.sort_values()

Int64Index([1890, 1891, 1892, 1893, 1894, 1895, 1896, 1897, 1898, 1899, 1900,
            1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910, 1911,
            1912, 1913, 1914, 1915, 1916, 1917, 1918, 1919, 1920, 1921, 1922,
            1923, 1924, 1925, 1926, 1927, 1928, 1929, 1930, 1931, 1932, 1933,
            1934, 1935, 1936, 1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944,
            1945, 1946, 1947, 1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955,
            1956, 1957, 1958, 1961, 1962, 1963, 1964, 1965, 1968, 1970, 1972,
            1980, 1995, 1997, 2000, 2009, 2013, 2014],
           dtype='int64')

In [64]:
weeks_on_chart_conflict = df[ df['Number_Week_on_Charts'] != df['CH'] ]
weeks_on_chart_conflict[weeks_on_chart_conflict["Year"] > 1990]

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week,First_Week_on_Charts,Number_Week_on_Charts,diff
271,2014,103,h,2014_069,28.0,18.0,0.0,1.0,15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,29,1.0
640,2013,67,h,2013_067,37.0,16.0,0.0,1.0,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,38,1.0
2510,2009,170,h,2009_170,26.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,1.0
5506,2000,51,y,2000_051,24.0,21.0,3.0,1.0,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,3.0
6448,1997,15,a,1997_015,65.0,60.0,19.0,2.0,2,t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,57,-8.0
7186,1995,80b,b,1995_080b,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1.0
7227,1995,120,a,1995_120,18.0,7.0,0.0,1.0,31,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19,19,1.0


In [66]:
weeks_on_chart_conflict["Year"].value_counts().head(20)

1953    441
1938    382
1937    379
1954    368
1940    361
1948    358
1941    339
1950    335
1939    329
1947    321
1951    318
1949    315
1952    315
1936    313
1934    308
1935    294
1946    285
1942    281
1945    266
1944    251
Name: Year, dtype: int64

In [69]:
# find how significant the difference is
weeks_on_chart_conflict["diff"] = (weeks_on_chart_conflict['Number_Week_on_Charts'] - weeks_on_chart_conflict['CH'])
weeks_on_chart_conflict.head()

C:\Users\ejwda\AppData\Local\Temp/ipykernel_28364/1435794705.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weeks_on_chart_conflict["diff"] = (weeks_on_chart_conflict['Number_Week_on_Charts'] - weeks_on_chart_conflict['CH'])


,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week,First_Week_on_Charts,Number_Week_on_Charts,diff
271,2014,103,h,2014_069,28.0,18.0,0.0,1.0,15,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29,29,1.0
640,2013,67,h,2013_067,37.0,16.0,0.0,1.0,14,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,38,38,1.0
2510,2009,170,h,2009_170,26.0,1.0,0.0,1.0,39,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,1.0
5506,2000,51,y,2000_051,24.0,21.0,3.0,1.0,8,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27,27,3.0
6448,1997,15,a,1997_015,65.0,60.0,19.0,2.0,2,t,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57,57,-8.0


In [70]:
weeks_on_chart_conflict["diff"].value_counts()

-1.0     3406
-2.0     1774
-3.0     1286
-4.0     1112
-5.0      802
-6.0      609
-7.0      461
-8.0      375
-10.0     302
-9.0      293
-11.0     266
-12.0     226
-13.0     184
-14.0     179
-15.0     138
-17.0     123
-16.0     113
-19.0      83
-18.0      76
-20.0      62
-21.0      46
-22.0      36
-24.0      31
-23.0      31
-25.0      29
-27.0      20
-26.0      13
 1.0       11
-28.0       8
-30.0       7
-29.0       6
-31.0       4
-34.0       3
 3.0        2
 2.0        2
-32.0       2
 7.0        1
 5.0        1
 12.0       1
-38.0       1
-33.0       1
-36.0       1
Name: diff, dtype: int64

In [96]:
year_list = weeks_on_chart_conflict.groupby("diff")['Year'].apply(set)
year_counts = year_list.apply(len)
year_min= year_list.apply(min)
year_max = year_list.apply(max)

In [97]:
summary = pd.DataFrame({'Year List':year_list, 'Year Count': year_counts, 'Min Year': year_min, 'Max Year':year_max})
summary

,Year List,Year Count,Min Year,Max Year
diff,,,,
-38.0,{1952},1,1952,1952
-36.0,{1943},1,1943,1943
-34.0,"{1943, 1951}",2,1943,1951
-33.0,{1944},1,1944,1944
-32.0,"{1948, 1951}",2,1948,1951
-31.0,"{1953, 1956}",2,1953,1956
-30.0,"{1952, 1940, 1943, 1948, 1951}",5,1940,1952
-29.0,"{1945, 1956, 1951}",3,1945,1956
-28.0,"{1956, 1931, 1943, 1944, 1945}",5,1931,1956


In [5]:
df_basics = df.iloc[:, :34]
df_basics.head()

,Year,Yearly Rank,Source,Prefix,CH,40,10,PK,High,Verified,...,Label/Number,Media,Stereo (55-68),Pic Sleeve,Genre,Comments,Written By,Temp 1,Date Entered,Date Peaked
0,2015,262,h,2015-001,25.0,1.0,0.0,1.0,39,NaN,...,Lichtdicht Digital,DD,NaN,NaN,NaN,NaN,C.Rehbein,1079,8/9/14,1/3/15
1,2015,298,h,2015-002,34.0,27.0,2.0,2.0,10,NaN,...,Decaydance/Island Digital,DD,NaN,NaN,NaN,NaN,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...",2467,9/27/14,2/7/15
2,2015,301,h,2015-003,32.0,25.0,10.0,1.0,7,NaN,...,Safehouse/Island Digital,DD,NaN,NaN,NaN,NaN,"N.Jonas,N.Lambrozza,S.Wilcox",2383,9/27/14,1/24/15
3,2015,304,h,2015-004,16.0,5.0,0.0,1.0,18,NaN,...,Epic Digital,DD,NaN,NaN,NaN,NaN,"M.Trainor,K.Kadish",583,9/27/14,5/16/15
4,2015,308,h,2015-005,20.0,0.0,0.0,1.0,44,NaN,...,Republic Nashville Digital,DD,NaN,NaN,NaN,NaN,"C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley",770,10/4/14,2/7/15


In [6]:
len(df_basics)

39982

In [7]:
df_basics["Prefix"].nunique()

39982

In [8]:
# list of writers
df_basics['Written By'].str.split(",", 10, expand=True)

,0,1,2,3,4,5,6,7,8,9,10
0,C.Rehbein,None,None,None,None,None,None,None,None,None,None
1,J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley,M.J.Fonseca,R.Kumari,J.Trantner,S.Vega,None,None
2,N.Jonas,N.Lambrozza,S.Wilcox,None,None,None,None,None,None,None,None
3,M.Trainor,K.Kadish,None,None,None,None,None,None,None,None,None
4,C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
39977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39979,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39980,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
selected_columns = [0, 10, 16, 24, 28, 30] + list(range(34, 125))

In [10]:
df_weekly = df.iloc[:,selected_columns]
df_weekly.head()

,Year,Artist,Track,Label/Number,Genre,Written By,1st Week,2nd Week,3rd Week,4th Week,...,82nd Week,83rd Week,84th Week,85th Week,86th Week,87th Week,88th Week,89th Week,90th Week,91st Week
0,2015,Milky Chance,Stolen Dance,Lichtdicht Digital,NaN,C.Rehbein,95,91,84,71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,Fall Out Boy,Centuries,Decaydance/Island Digital,NaN,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...",22,53,55,54,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,Nick Jonas,Jealous,Safehouse/Island Digital,NaN,"N.Jonas,N.Lambrozza,S.Wilcox",78,80,58,40,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,Meghan Trainor,Dear Future Husband,Epic Digital,NaN,"M.Trainor,K.Kadish",94,95,93,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,Florida Georgia Line,Sun Daze,Republic Nashville Digital,NaN,"C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley",83,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_weekly_T = df_weekly.transpose()

In [174]:
headers = df_weekly_T.iloc[2,:]
len(headers)

39982

In [173]:
len(set(headers))

30170

In [175]:
single_items = []
duplicates = []
for x in headers:
    if x in single_items:
        duplicates.append(x)
    else:
        single_items.append(x)
        
len(duplicates)

9812

In [177]:
single_duplicates = set(duplicates)
len(single_duplicates)

5368

In [13]:
df_weekly_T.columns = headers
df_weekly_T.head(20)

Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,Pat Kelly as a Police Justice,Little 'Liza Loves You,"Home, Sweet Home",The Picture Turned to the Wall,Uncle Jefferson,Pat Kelly on a Spree,Pat Kelly's Plea in His Own Defense,Semper Fidelis,Washington Post,The Thunderer
Year,2015,2015,2015,2015,2015,2015,2015,2015,2015,2015,...,1891,1891,1891,1891,1891,1891,1891,1890,1890,1890
Artist,Milky Chance,Fall Out Boy,Nick Jonas,Meghan Trainor,Florida Georgia Line,Alesso,Tim McGraw,Ed Sheeran,Kid Ink,"Weeknd, The",...,Dan Kelly,Len Spencer,John Yorke Atlee,George J. Gaskin,Billy Golden,Dan Kelly,Dan Kelly,U.S. Marine Band,U.S. Marine Band,U.S. Marine Band
Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,Pat Kelly as a Police Justice,Little 'Liza Loves You,"Home, Sweet Home",The Picture Turned to the Wall,Uncle Jefferson,Pat Kelly on a Spree,Pat Kelly's Plea in His Own Defense,Semper Fidelis,Washington Post,The Thunderer
Label/Number,Lichtdicht Digital,Decaydance/Island Digital,Safehouse/Island Digital,Epic Digital,Republic Nashville Digital,Refune Digital,McGraw Digital (Big Machine),Atlantic Digital,Tha Alumni Group/88 Classic Digital,XO Digital (Republic),...,Ohio (No Number),Columbia (No Number),Columbia (No Number),North American (No Number),Columbia (No Number),Ohio (No Number),Ohio (No Number),Columbia (No Number),Columbia (No Number),Columbia (No Number)
Genre,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Written By,C.Rehbein,"J.R.Rotem,P.V.Stump,P.Wentz,J.Trohman,A.Hurley...","N.Jonas,N.Lambrozza,S.Wilcox","M.Trainor,K.Kadish","C.R.Barlowe,J.Frasure,S.Buxton,T.Hubbard,B.Kelley","A.Lindblad,T.Lo,D.Bowie,B.Eno","H.Lindsey,M.Green,T.Verges","E.Sheeran,A.Wadge","B.T.Collins,M.S.Eriksen,T.E.Hermansen,M.A.Hoib...","A.Tesfaye,B.Diehl,J.Quenneville,A.Balshe,D.Sch...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st Week,95,22,78,94,83,89,97,69,100,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Week,91,53,80,95,NaN,80,84,79,100,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3rd Week,84,55,58,93,NaN,78,76,75,88,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4th Week,71,54,40,NaN,NaN,72,65,84,87,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [178]:
df_weekly_T[single_duplicates]

C:\Users\ejwda\AppData\Local\Temp/ipykernel_28364/4159731507.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df_weekly_T[single_duplicates]


Track,NaN,NaN,Mandy,Mandy,Mandy,Mandy,Little By Little,Little By Little,Little By Little,Out of This World,...,Around the Corner,Around the Corner,Around the Corner,Many Times,Many Times,Irene,Irene,What's Going On,What's Going On,What's Going On
Year,1924,1922,1975,1920,1919,1919,1985,1957,1957,1945,...,1930,1930,1930,1953,1953,1920,1920,2001,1987,1971
Artist,See 1924_133,VOID,Barry Manilow,Ben Selvin & His Orchestra,Van & Schenck,Shannon Four,Robert Plant,Micki Marlo,Nappy Brown,Jo Stafford,...,Tom Gerun & His Orchestra,Leo Reisman & His Orchestra,Ben Selvin & His Orchestra,Eddie Fisher,Percy Faith & His Orchestra,Edith Day,Joseph C. Smith's Orchestra,Artists Against AIDS,Cyndi Lauper,Marvin Gaye
Track,NaN,NaN,Mandy,Mandy,Mandy,Mandy,Little By Little,Little By Little,Little By Little,Out of This World,...,Around the Corner,Around the Corner,Around the Corner,Many Times,Many Times,Irene,Irene,What's Going On,What's Going On,What's Going On
Label/Number,NaN,NaN,Bell 45613,Victor 18614,Columbia 2780,Victor 18605,Es Paranza 7-99644,ABC-Paramount 9762,Savoy 1506,Capitol 191,...,Brunswick 4829,Victor 22459,Columbia 2221,RCA Victor 5453,Columbia 40078,Victor 45176,Victor 35695,Play-Tone 79670,Portrait 37-06970,Tamla 54201
Genre,NaN,NaN,Rock,NaN,NaN,NaN,Rock,Rock,Blues,Vocal,...,NaN,NaN,NaN,Vocal,Easy,NaN,NaN,Rap,Rock,Rock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90th Week,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [183]:
pd.Series(list(duplicates)).value_counts().head(30)


White Christmas             33
You                         18
I Love You                  16
Hold On                     15
Always                      15
Angel                       14
Star Dust                   14
Forever                     12
Hold Me                     12
Smile                       12
Happy                       12
Love Me                     12
Body and Soul               12
Stay                        12
Without You                 12
Crazy                       11
Runaway                     11
Cry                         11
St. Louis Blues             11
Home                        10
Alexander's Ragtime Band    10
Tell Me Why                 10
Baby Face                   10
Down Yonder                 10
If                          10
I Need You                  10
Sunshine                     9
Harbor Lights                9
Call Me                      9
After You've Gone            9
dtype: int64

In [156]:
df_main = df_weekly_T.iloc[6:, :]
df_main.head()

Track,Stolen Dance,Centuries,Jealous,Dear Future Husband,Sun Daze,Heroes (We Could Be),Shotgun Rider,Thinking Out Loud,Body Language,Often,...,Pat Kelly as a Police Justice,Little 'Liza Loves You,"Home, Sweet Home",The Picture Turned to the Wall,Uncle Jefferson,Pat Kelly on a Spree,Pat Kelly's Plea in His Own Defense,Semper Fidelis,Washington Post,The Thunderer
1st Week,95,22,78,94,83,89,97,69,100,97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Week,91,53,80,95,NaN,80,84,79,100,95,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3rd Week,84,55,58,93,NaN,78,76,75,88,99,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4th Week,71,54,40,NaN,NaN,72,65,84,87,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5th Week,66,42,28,NaN,NaN,73,66,68,80,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
df_main.dtypes

Track
Stolen Dance                           float64
Centuries                              float64
Jealous                                 object
Dear Future Husband                     object
Sun Daze                                object
                                        ...   
Pat Kelly on a Spree                    object
Pat Kelly's Plea in His Own Defense     object
Semper Fidelis                          object
Washington Post                         object
The Thunderer                           object
Length: 39982, dtype: object

In [161]:
df_main['Jealous']

Track,Jealous,Jealous,Jealous,Jealous,Jealous
1st Week,78,95,NaN,NaN,NaN
2nd Week,80,88,NaN,NaN,NaN
3rd Week,58,80,NaN,NaN,NaN
4th Week,40,78,NaN,NaN,NaN
5th Week,28,77,NaN,NaN,NaN
...,...,...,...,...,...
87th Week,NaN,NaN,NaN,NaN,NaN
88th Week,NaN,NaN,NaN,NaN,NaN
89th Week,NaN,NaN,NaN,NaN,NaN
90th Week,NaN,NaN,NaN,NaN,NaN


In [160]:
pd.to_numeric(df_main['Jealous'])

TypeError: arg must be a list, tuple, 1-d array, or Series

In [158]:
for col in df_main.columns:
    df_main[col] = pd.to_numeric(df_main[col])

df_main.plot()

C:\Users\ejwda\AppData\Local\Temp/ipykernel_28364/79133386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main[col] = pd.to_numeric(df_main[col])
C:\Users\ejwda\AppData\Local\Temp/ipykernel_28364/79133386.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_main[col] = pd.to_numeric(df_main[col])


TypeError: arg must be a list, tuple, 1-d array, or Series

In [143]:
cols=[]
cols.append("Hello") if not 1.0 in df_main['Stolen Dance'].values else "Goodbye"

In [144]:
cols=[]
for col in df_main.columns:
    cols.append(col) if 1.0 in df_main[col].values else None
    # print(row[1].values)

In [148]:
len(df_main.columns)
# df_plot[cols]

50